```
Notebook Script for update grade control blast hole data
```

# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from pathlib import Path
import csv
import shutil

## Setup variables

In [ ]:
date_file = '20251126'
file_name = "bh_519_01_midlusong_217-5_20251126.csv"
dp_id = "01_midlusong_217.5"
bench = '217.5S'

# Set paths

In [ ]:
base = Path("../../mr_directory/11_grade_control").resolve()

survey_holes = base / f"01_survey_holes/{date_file[:4]}/{date_file[:6]}/{date_file}"
logbook = base / f"02_sampling/logbook/{date_file[:4]}/{date_file[:6]}"
grade_samples = base / f"03_grade_samples/{date_file[:4]}/{date_file[:6]}/{date_file}"

paths = {
    "Survey holes": survey_holes,
    "Logbook": logbook,
    "Grade samples": grade_samples,
}

for name, p in paths.items():
    p = Path(p)
    print(f"{name}: {p} -> {'OK' if p.exists() else 'MISSING'}")

In [ ]:
drill_plan = survey_holes / "drill_plan" / file_name

if not drill_plan.is_file():
    raise FileNotFoundError(f"Missing drill plan file: {drill_plan}")

drill_plan

In [ ]:
loaded_holes = survey_holes / "loaded_holes" / file_name

if not loaded_holes.is_file():
    raise FileNotFoundError(f"Missing drill plan file: {loaded_holes}")

loaded_holes

# Initial validation of drill plan and loaded holes

In [ ]:
def clean_hole_id(x):
    try:
        # Convert float-like strings safely
        f = float(x)
        if f.is_integer():
            return int(f)
    except (ValueError, TypeError):
        pass
    return x

## Drill Plan

In [ ]:
df_dp = pd.read_csv(drill_plan, skiprows=2, index_col=False, header=None)
df_dp.columns = ['string', 'y', 'x', 'z', 'd1', 'hole_id', 'd3', 'd4']
df_dp = df_dp[df_dp["string"] != 0]

df_dp["hole_id"] = df_dp["hole_id"].apply(clean_hole_id)

df_dp["hole_id"] = df_dp["hole_id"].astype(str).str.strip()
df_dp

## Loaded holes

In [ ]:
df_lh = pd.read_csv(loaded_holes, skiprows=2, index_col=False, header=None)
df_lh.columns = ["string", "y", "x", "z", "hole_id", "d2", "d3", "d4"]
df_lh = df_lh[df_lh["string"] != 0]

df_lh["hole_id"] = df_lh["hole_id"].apply(clean_hole_id)

df_lh["hole_id"] = df_lh["hole_id"].astype(str).str.strip()
df_lh   

### Nearest drill plan hole_id

In [ ]:
tree = cKDTree(df_dp[['x', 'y']].values)
distances, indices = tree.query(df_lh[['x', 'y']].values)

df_lh['nearest_hole_id'] = df_dp.iloc[indices]['hole_id'].values
df_lh['distance_to_nearest'] = distances

df_lh


# Merge loaded holes with drill plan

In [ ]:
def hole_sort_key(x):
    try:
        return (0, int(float(x)))  # numeric first
    except (ValueError, TypeError):
        return (1, str(x))  # then text

In [ ]:
merged_df = pd.merge(df_lh, df_dp, on="hole_id", how="outer", suffixes=("_lh", "_dp"))
merged_df = merged_df.sort_values(by="hole_id", key=lambda s: s.map(hole_sort_key))
merged_df

In [ ]:
merged_df["d3"] = merged_df.apply(
    lambda row: row["d3_dp"] if row["d3_dp"] == row["d3_lh"] else row["d3_dp"], axis=1
)
merged_df["d4"] = merged_df.apply(
    lambda row: row["d4_dp"] if row["d4_dp"] == row["d4_lh"] else row["d4_dp"], axis=1
)

merged_df = merged_df.drop(
    columns=[
        "d3_dp",
        "d3_lh",
        "d4_dp",
        "d4_lh",
    ]
)
merged_df

## Offset Computation

Distance formula:

$d = \sqrt{(x_{2} - x_{1})^{2} + (y_{2} - y_{1})^{2}}$

Azimuth formula:

$\theta = \arctan2(\Delta x,\; \Delta y)$

Radians to Degrees conversion:

$\theta_{\text{deg}} = \theta \times \frac{180}{\pi}$

In [ ]:
# Delta x and Delta y
merged_df["dx"] = merged_df["x_lh"] - merged_df["x_dp"]
merged_df["dy"] = merged_df["y_lh"] - merged_df["y_dp"]

# Distance (Euclidean)
merged_df["distance"] = np.sqrt(merged_df["dx"] ** 2 + merged_df["dy"] ** 2)

# Azimuth (clockwise from north)
merged_df["azimuth"] = np.degrees(np.arctan2(merged_df["dx"], merged_df["dy"]))
merged_df["azimuth"] = merged_df["azimuth"] % 360

merged_df = merged_df.drop(columns=["dx", "dy"])           
merged_df

## Validation

### Nearest hole_id validation

In [ ]:
filtered_df = merged_df[merged_df['hole_id'] != merged_df['nearest_hole_id']]
filtered_df

In [ ]:
invalid_hole_ids = filtered_df[filtered_df['x_dp'].notna() & filtered_df['x_lh'].notna()]
invalid_hole_ids

## Missing rows

### Loaded Holes ✅ | Drill Plan ❌

Holes below are additional holes

In [ ]:
missing_in_dp = filtered_df[filtered_df["x_dp"].isna()]
missing_in_dp

### Loaded Holes ❌ | Drill Plan ✅

Holes below are not drilled

In [ ]:
missing_in_lh = filtered_df[filtered_df["x_lh"].isna()]
missing_in_lh

# Cleaned merged data

Make sure the survey data is cleaned before running script below

In [ ]:
merged_df

# Import Logbook.csv

In [ ]:
df_lb = pd.read_csv(f"{logbook}/logbook.csv")
df_lb = df_lb[df_lb["Drill Plan"] == dp_id]

df_lb

## Filter Inserts

In [ ]:
mask = (
    df_lb["Sample ID"]
    .astype(str)
    .str.contains(r"\b(?:CR|DUP|BLANK)\b", case=False, na=False)
)

In [ ]:
inserts = df_lb[mask].copy()
inserts

In [ ]:
df_lb = df_lb[~mask]

df_lb

## Final Cu

In [ ]:
df_lb['Cu_final'] = df_lb['Cu_Reassay'].fillna(df_lb['Cu_Orig'])
df_lb = df_lb.drop(columns=['Cu_Orig', 'Cu_Reassay'])

df_lb

# Final dataframe | Merged survey and assay data

In [ ]:
df_final = pd.merge(merged_df, df_lb, left_on='hole_id', right_on='Sample ID', how='outer')

df_final = df_final[df_final['Unique ID'].notna()]
df_final['d4'] = date_file
df_final['Cu_final'] = pd.to_numeric(df_final['Cu_final'])
df_final['Lab Number'] = df_final['Lab Number'].astype(int)

df_final

## Sort by Unique ID

In [ ]:
df_final = df_final.sort_values("Lab Number").reset_index(drop=True)
df_final

## Validation

### Check Unique ID sequence

In [ ]:
inserts

In [ ]:
# samp_id = df_final["num_part"].astype(int)

# if not (samp_id.diff().dropna() == 1).all():
#     raise ValueError("Lab Number is NOT sequential.")

### Check samples with no loaded holes

In [ ]:
cols = ['y_lh', 'x_lh', 'z_lh']
missing_hole_id = df_final[df_final[cols].isna().any(axis=1)]
missing_hole_id

Edit loaded holes string file and add loaded holes points based on the drill plan.
Make sure to drape the point on Survey Progress to use actual coordinates.

In [ ]:
if not missing_hole_id.empty:
    holes_str = ', '.join(missing_hole_id['hole_id'].astype(str))
    raise ValueError(f"Error: Samples with missing coordinates found for the following holes: {holes_str}")

### Check duplicate IDs

In [ ]:
duplicates_mask = df_final["hole_id"].duplicated(keep=False)
df_duplicates = df_final[duplicates_mask]

df_duplicates

#### Handle duplicates | Average the Values
If the differences are minor and you want one representative value per hole
- Compute the mean (or weighted mean if sample lengths differ).
- Use the averaged value in your block model.

This reduces bias from multiple entries of the same hole.

##### Remove duplicates

In [ ]:
df_final = df_final[~duplicates_mask]

##### Calculate

In [ ]:
cols_to_numeric = ['Cu_final', 'CuO_Soluble', 'Au', 'Ag']

df_duplicates[cols_to_numeric] = df_duplicates[cols_to_numeric].apply(pd.to_numeric, errors='coerce')

df_duplicates

In [ ]:
cols_to_concat = ['Unique ID', 'Lab Number', 'Sample Description', 'file_name'] 

sample_ids = df_duplicates.groupby('hole_id')[cols_to_concat].agg(
    lambda x: ','.join(x.dropna().astype(str))
).reset_index()

sample_ids

In [ ]:
avg_df = df_duplicates.groupby('hole_id', as_index=False).agg({
    'Cu_final': 'mean',
    'CuO_Soluble': 'mean',
    'Au': 'mean',
    'Ag': 'mean',
})

avg_df

In [ ]:
averaged_samples = sample_ids.merge(avg_df, on='hole_id')
averaged_samples

In [ ]:
df_clean = df_duplicates.drop(columns=cols_to_concat).drop_duplicates(subset='hole_id')
df_clean = df_clean.drop(columns=['Cu_final', 'CuO_Soluble', 'Au', 'Ag'] + cols_to_concat, errors='ignore') 
df_clean = df_clean.merge(averaged_samples, on='hole_id')
df_clean

##### Final dataframe with averaged grades

In [ ]:
df_final = pd.concat([df_final, df_clean], ignore_index=True)
df_final

# Convert to grade samples

In [ ]:
# Filter columns
df_final['BHID'] = df_final['d4'] + "_" + bench + "_" + df_final['hole_id']

filtered_cols = ['string_lh', 'y_lh', 'x_lh', 'z_lh', 'Cu_final', 'hole_id', 'd3', 'd4', 'BHID',  'Drill Plan', 'z_dp', 'Unique ID', 'Sample ID','CuO_Soluble', 'Au', 'Ag']
df_grade_samples = df_final[filtered_cols].copy()

bins = [0, 0.119, 0.179, 0.249, 0.299, 0.399, 0.599, 999]
labels = [1, 2, 3, 4, 5, 6, 7]

# Create the 'string_lh' column based on 'Cu_final'
df_grade_samples['string_lh'] = pd.cut(
    df_grade_samples['Cu_final'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

gs_file = grade_samples / file_name
gs_db_file = gs_file.with_name(gs_file.stem + "_db" + gs_file.suffix)

df_grade_samples.to_csv(gs_file, index=False)
df_grade_samples.to_csv(gs_db_file, index=False)
df_grade_samples

# Convert back to string files

In [ ]:
# Read original CSV
with open(gs_file, newline='') as f:
    reader = list(csv.reader(f))

# Remove the first row
reader = reader[1:]

# Add rows at top and bottom
new_rows = [
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0]
] + reader + [
    [0, 0, 0, 0],
    [0, 0, 0, 0, "END"]
]

# Write back to CSV
with open(gs_file, "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(new_rows)


shutil.copy(gs_file, f"{gs_file.parent}/{gs_file.stem}.str")